## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Huffman:
    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        pass

    def decode(self, code: str, coding_dict: dict[str, str]):
        pass

# Алгоритм LZW

In [ ]:
from timeit import default_timer as timer
def measure(text, obj):
    start = timer()
    encoded = obj.encode(text)
    end = timer()
    print('Час кодування якогось тексту ---',end - start, 'секунд.')
    start = timer()
    decoded = obj.decode(encoded[0], encoded[1])
    end = timer()
    print('Час декодування якогось тексту ---', end - start,'секунд.')
    print('Ступінь стиснення --- ', round((len(encoded[0])/len(decoded))*100, 2),f'%.; {len(encoded[0])=}, {len(decoded)=}.\n')
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        '''
        1) створюється словник з усіма стрічками довжини 1
        2) знаходиться найдовша стрічка у словнику, що співпадає з поточним інпутом
        3) посилається на аутпут індекс стрічки у словнику та прибирається стрічка з інпуту
        4) у словник додається стрічка разом із наступним символом у інпуті
        5) повторюється крок 2, поки довжина інпуту більше нуля
        '''
        output = []
        # lst = [chr(x).upper() for x in range(97, 123)]
        lst = [x for x in sorted(set(text))]
        def find(smth):
            for i in range(len(smth), 0, -1):
                for j in lst[::-1]:
                    if smth[:i] == j:
                        if smth[:i+1] not in lst:
                            lst.append(smth[:i+1])
                        output.append(lst.index(j)+1)
                        smth = smth.replace(j, '', 1)
                        return smth
        while text:
            text = find(text)
        output = ','.join(str(x) for x in output)
        return output, lst


    def decode(self, code: str, coding_dict: list) -> str:
        '''
        1) зчитується закодований символ
            1.1) якщо є у словнику, то
                1.1.1) сума попередньої стрічка в аутпуті та першого символа поточної стрічки додається до словника
                1.1.2) відповідна стрічка у словнику посилається на аутпут
            1.2) якщо нема у словнику, то
                1.2.1) сума попередньої стрічки в аутпуті та першого символа цієї стрічки додається до словника
                1.2.2) та ж сума посилається на аутпут
        2) повторюється крок 2 до кінця стрічки інпуту
        '''
        code = [int(x) for x in code.split(',')]
        def find(code):
            output = []
            for i in range(len(code)):
                if i == 0:
                    output.append(coding_dict[code[i]-1])
                elif code[i]-1<len(coding_dict):
                    w = coding_dict[code[i]-1]
                    coding_dict.append(output[-1] + w[0])
                    output.append(w)
                else:
                    v = output[-1] + output[-1][0]
                    coding_dict.append(v)
                    output.append(v)
            return output
        return "".join(find(code))

obj = LZW()
print('abacababacabc' == obj.decode(obj.encode('abacababacabc')[0], obj.encode('abacababacabc')[1]))
print('adbbadbcabdbadadcbda' == obj.decode(obj.encode('adbbadbcabdbadadcbda')[0], obj.encode('adbbadbcabdbadadcbda')[1]))


initial_msg = 'abacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabaca\
babacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababac\
abcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabc'
compressed = obj.encode(initial_msg)
decompressed = obj.decode(compressed[0], compressed[1])
assert decompressed == initial_msg
print(len(compressed[0]), len(decompressed))

measure('abdcabdcabdcabcdbaacddbccdcbdbabababdcabbaaaabbcdbdcabcdbbbcbcbcaabdcadabdabcbdcadbacdcbadabcdcdabcbacdbcabdbacdbbcddcbdcdcdbaadcbaccbdbabcabbabaadbcabdcabdbabdcabdc', obj)



measure('Anyways uhm... I bought a whole bunch of shungite, rocks, \
do you know what shungite is? Anybody know what shungite is. No, no Suge Knight, \
I think hes locked up in prison. Talking shungite. Anyways, its a 2 billion year old like rock, \
stone that protects against frequencies and unwanted frequencies that may be traveling in the air. \
So thats my story. I bought a whole bunch of stuff, put them around the la casa. Little pyramids. Stuff like that.', obj)



measure("На початку Бог створив Небо та землю.\
2 А земля була пуста та порожня, і темрява була над безоднею, і Дух Божий ширяв над поверхнею води.\
3 І сказав Бог: Хай станеться світло! І сталося світло.\
4 І побачив Бог світло, що добре воно", obj)#, і Бог відділив світло від темряви.\
# 5 І Бог назвав світло: День, а темряву назвав: Ніч. І був вечір, і був ранок, день перший.\
# 6 І сказав Бог: Нехай станеться твердь посеред води, і нехай відділяє вона між водою й водою.\
# 7 І Бог твердь учинив, і відділив воду, що під твердю вона, і воду, що над твердю вона. І сталося так.\
# 8 І назвав Бог твердь Небо. І був вечір, і був ранок день другий.\
# 9 І сказав Бог: Нехай збереться вода з-попід неба до місця одного, і нехай суходіл стане видний. І сталося так.\
# 10 І назвав Бог суходіл: Земля, а місце зібрання води назвав: Море. І Бог побачив, що добре воно.\
# 11 І сказав Бог: Нехай земля вродить траву, ярину, що насіння вона розсіває, дерево овочеве, що за родом своїм плід приносить, що в ньому насіння його на землі. І сталося так.\
# 12 І земля траву видала, ярину, що насіння розсіває за родом її, і дерево, що приносить плід, що насіння його в нім за родом його. І Бог побачив, що добре воно.\
# 13 І був вечір, і був ранок, день третій.\
# 14 І сказав Бог: Нехай будуть світила на тверді небесній для відділення дня від ночі, і нехай вони стануть знаками, і часами умовленими, і днями, і роками.\
# 15 І нехай вони стануть на тверді небесній світилами, щоб світити над землею. І сталося так.\
# 16 І вчинив Бог обидва світила великі, світило велике, щоб воно керувало днем, і світило мале, щоб керувало ніччю, також зорі.\
# 17 І Бог умістив їх на тверді небесній, щоб світили вони над землею,\
# 18 і щоб керували днем та ніччю, і щоб відділювали світло від темряви. І Бог побачив, що це добре.\
# 19 І був вечір, і був ранок, день четвертий.\
# 20 І сказав Бог: Нехай вода вироїть дрібні істоти, душу живу, і птаство, що літає над землею під небесною твердю.\
# 21 І створив Бог риби великі, і всяку душу живу плазуючу, що її вода вироїла за їх родом, і всяку пташину крилату за родом її. І Бог побачив, що добре воно.\
# 22 І поблагословив їх Бог, кажучи: Плодіться й розмножуйтеся, і наповнюйте воду в морях, а птаство нехай розмножується на землі!\
# 23 І був вечір, і був ранок, день п'ятий.\
# 24 І сказав Бог: Нехай видасть земля живу душу за родом її, худобу й плазуюче, і земну звірину за родом її. І сталося так.\
# 25 І вчинив Бог земну звірину за родом її, і худобу за родом її, і все земне плазуюче за родом його. І бачив Бог, що добре воно.\
# 26 І сказав Бог: Створімо людину за образом Нашим, за подобою Нашою, і хай панують над морською рибою, і над птаством небесним, і над худобою, і над усею землею, і над усім плазуючим, що плазує по землі.\
# 27 І Бог на Свій образ людину створив, на образ Божий її Він створив, як чоловіка та жінку створив їх.\
# 28 І поблагословив їх Бог, і сказав Бог до них: Плодіться й розмножуйтеся, і наповнюйте землю, оволодійте нею, і пануйте над морськими рибами, і над птаством небесним, і над кожним плазуючим живим на землі!\
# 29 І сказав Бог: Оце дав Я вам усю ярину, що розсіває насіння, що на всій землі, і кожне дерево, що на ньому плід деревний, що воно розсіває насіння, нехай буде на їжу це вам!\
# 30 І земній усій звірині і всьому птаству небесному, і кожному, що плазує по землі, що душа в ньому жива, уся зелень яринна на їжу для них. І сталося так.\
# 31 І побачив Бог усе, що вчинив. І ото, вельми добре воно! І був вечір, і був ранок, день шостий.", obj)
print("Чим менше повторень символів, тим довше закодоване повідомлення та час виконання алгоритму.")


True
True
180 312
Час кодування якогось тексту --- 0.02297765999992407 секунд.
Час декодування якогось тексту --- 6.969200012463261e-05 секунд.
Ступінь стиснення ---  113.33 %.; len(encoded[0])=187, len(decoded)=165.

Час кодування якогось тексту --- 1.345512731999861 секунд.
Час декодування якогось тексту --- 0.00040361199990002206 секунд.
Ступінь стиснення ---  203.79 %.; len(encoded[0])=913, len(decoded)=448.

Час кодування якогось тексту --- 0.5890526950001913 секунд.
Час декодування якогось тексту --- 0.0002830789999279659 секунд.
Ступінь стиснення ---  225.44 %.; len(encoded[0])=514, len(decoded)=228.

Чим менше повторень символів, тим довше закодоване повідомлення та час виконання алгоритму.


# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [ ]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    @staticmethod
    def find_second_last_occurrence(string, str_to_find):
        """Допоміжна функція для знаходження офсету, знаходить 2ге з кінця входження значення"""
        # Знаходимо всі індекси, де починається підрядок
        indices = [i for i in range(len(string) - len(str_to_find) + 1) \
    if string.startswith(str_to_find, i)]
        # Якщо знайдено 2 або більше входження, повертаємо передостаннє
        if len(indices) >= 2:
            return indices[-2]
        else:
            return -1

    def encode(self, text: str) -> str:
        """lz77 закодування"""
        all_text = text
        result, buffer = [], []
        ofset, lenght, nexxt = 0, 0, ''
        ind = 0
        while text:
            if all_text[ind] in buffer:
                buffer_lenght_befor = len(buffer) #якщо буфер ще не заповнений повністю
                ofset = buffer_lenght_befor - ''.join(buffer).rfind(all_text[ind])
                buffer.append(all_text[ind][-1]) #щоб цей прикол з повторами одразу ловити
                text = text[1:]
                matched = all_text[ind] #ця літера є, того змечили її
                for i, el in enumerate(text): #Шукаємо найкращий метч
                    if matched+el in "".join(buffer):
                        matched += el
                        buffer.append(el)
                    else:
                        buffer.append(el)
                        ofset = ''.join(buffer).rfind(matched) - \
                            LZ77.find_second_last_occurrence(''.join(buffer), matched)
                        #на випадок, що в нас було кілька варіантів "початку", але обираємо той,
                        # що дозволяє за один раз більше закодити
                        if len(buffer) > self.buffer_size:
                            buffer = buffer[-self.buffer_size:]
                        lenght, nexxt = len(matched), el
                        nexxt = el
                        if i == len(text) - 1:
                            nexxt = " "
                        text = text[len(matched):]
                        result.append((ofset, lenght, nexxt))
                        ind += len(matched) +1
                        break
            else: #випадок першого входження літери
                buffer.append(all_text[ind])
                ofset, lenght, nexxt = 0, 0, all_text[ind]
                result.append((ofset, lenght, nexxt))
                text, buffer = text[1:], buffer[-self.buffer_size:]
                ind += 1
        return result

    def decode(self, code: str) -> str:
        """Алгоритм декожування до lz77"""
        result = []
        for el in code:
            ofset, lenght, neext = el
            if ofset == 0: #перше входження літери
                result.append(neext)
            elif ofset == lenght: #звичайний випадок
                result.append(''.join(result)[-ofset:] + neext)
            elif ofset > lenght: #звичайний випадок
                result.append(''.join(result)[-ofset:-ofset+lenght] + neext)
            else: #ця наша найприкріша ситуація
                timed = "".join(result)[-ofset]*(lenght//ofset)
                timed += timed[:-(lenght%ofset)] + neext
                result.append(timed)
        result = "".join(result)
        return result

In [ ]:
from timeit import default_timer as timer
def measure(text, obj):
    start = timer()
    encoded = obj.encode(text)
    end = timer()
    print('Час кодування якогось тексту ---',end - start, 'секунд.')
    start = timer()
    decoded = obj.decode(encoded)
    end = timer()
    print('Час декодування якогось тексту ---', end - start,'секунд.')
    print('Ступінь стиснення --- ', round(len(encoded)/len(decoded)*100, 2),'%.\n', len(decoded), len(encoded))

buffer = LZ77(5)
our_text = 'Карл у Клари украв коралі, а Клара у Карла вкрала кларнет'
codded = buffer.encode(our_text)
our_text == buffer.decode(codded)
# our_text == decoding(codded)

measure('На початку Бог створив Небо та землю.\
2 А земля була пуста та порожня, і темрява була над безоднею, і Дух Божий ширяв над поверхнею води.\
3 І сказав Бог: Хай станеться світло! І сталося світло.\
4 І побачив Бог світло, що добре воно, і Бог відділив світло від темряви.', buffer)

measure('aduifynuiujidufuadfuanfikafopqorpqorqeruhafnsmiurokoiauaokcakjhipjnbyclqgavyfcgbutbagrvvt8', buffer)

Час кодування якогось тексту --- 0.0007657630001176585 секунд.
Час декодування якогось тексту --- 0.0002388349998909689 секунд.
Ступінь стиснення ---  83.33 %.
 264 220
Час кодування якогось тексту --- 0.00023812400013412116 секунд.
Час декодування якогось тексту --- 3.5796000020127394e-05 секунд.
Ступінь стиснення ---  75.56 %.
 90 68


# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass